In [1]:
import gurobipy as gp
from gurobipy import GRB
import math

# Regression
Compare optimal answer found through gurobi with closed form solution.

In [2]:
def calculate_optimal_gurobi(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=100):
    m = gp.Model('box_constraint_test')
    m.setParam( 'OutputFlag', False )
    m.setParam("MIPGap", 1e-10)
    m.setParam("MIPGapAbs", 1e-6)
    # Variables
    
    Bi = m.addVar(lb=lb, ub=ub, vtype=GRB.CONTINUOUS, name="Bi")
    Bi_supp = m.addVar(vtype=GRB.BINARY, name = '1[Bi]')
    Bi_sign = m.addVar(vtype=GRB.BINARY)
    Bi_pos = m.addVar(lb=0, ub=ub, vtype=GRB.CONTINUOUS)
    Bi_neg = m.addVar(lb=0, ub=-lb, vtype=GRB.CONTINUOUS)
    
    #Constraints
    m.addConstr(Bi_pos <= M*Bi_sign)
    m.addConstr(Bi_neg <= M*(1-Bi_sign))
    m.addConstr(Bi == Bi_pos - Bi_neg)

    m.addConstr(Bi >= -M*Bi_supp)
    m.addConstr(Bi <= M*Bi_supp)
    
    #Cost
    m.setObjective(0.5*(B - Bi)*(B - Bi) 
                   + l0*Bi_supp
                   + l1*(Bi_pos + Bi_neg) 
                   + l2*Bi*Bi, GRB.MINIMIZE)
    
    m.optimize()
    
    for v in m.getVars():
        if v.varName == "Bi":
            return v.x

In [3]:
def cost(B, Bi, l0, l1, l2):
    return 1/2*(B - Bi)**2 + l0*(Bi!=0) + l1*abs(Bi) + l2*Bi**2

def box(x, lb, ub):
    if x > ub:
        return ub
    elif x <= lb:
        return lb
    else:
        return x
    
def sign(x):
    if x < 0:
        return -1
    return 1


def calculate_closed_form(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=None):
    Bi = B # Attempt to make Bi, B. This is solved for iteratively using other approaches
    
    if (abs(Bi) - l1)/(1 + 2*l2) < math.sqrt(2*l0/(1 + 2*l2)):
        return 0
    
    elif (abs(Bi) - l1)/(1 + 2*l2) >= math.sqrt(2*l0/(1 + 2*l2)):
        
        delta = math.sqrt(((abs(Bi) - l1)**2 - 2*l0*(1 + 2*l2)))
        low = sign(Bi)*(abs(Bi) - l1)/(1+2*l2) - delta/(1+2*l2)
        high = sign(Bi)*(abs(Bi) - l1)/(1+2*l2) + delta/(1+2*l2)
        x = box(sign(Bi)*(abs(Bi) - l1)/(1+2*l2), lb, ub)
        if low <= x <= high:
            return x
        else:
            return 0
        
    
def CDL0(l0=None, B=None, lb=None, ub=None, M=None):
    x = box(B, lb, ub)
    
    if abs(B) < math.sqrt(2*l0):
        return 0
    
    delta = math.sqrt(B**2 - 2*l0)
    if not (B - delta <= x <= B + delta):
        return 0
    else:
        return x
    

def CDL02(l0=None, B=None, lb=None, ub=None, M=None):
    x = box(B, lb, ub)   
    delta = math.sqrt(abs(B**2 - 2*l0))    
    return x * (abs(B) >= math.sqrt(2*l0) and (B - delta <= x <= B + delta))

In [4]:
import hypothesis
from hypothesis import given, settings, assume, example
from hypothesis.strategies import floats
import numpy as np

# Compare values

In [5]:
@given(l0=floats(0, 10), l1 = floats(0, 10), l2 = floats(0, 10),
       B=floats(-10, 10), lb=floats(-1, 0), ub=floats(0, 1))
@settings(max_examples=10000, deadline=None)
@example(l0=0, l1=0, l2=0, B=1, lb=-1, ub=1)
@example(l0=0, l1=0, l2=10, B=1, lb=-1, ub=1)
@example(l0=0, l1=10, l2=0, B=1, lb=-1, ub=1)
@example(l0=0, l1=10, l2=10, B=1, lb=-1, ub=1)
@example(l0=10, l1=0, l2=10, B=1, lb=-1, ub=1)
@example(l0=10, l1=10, l2=0, B=1, lb=-1, ub=1)
@example(l0=10, l1=10, l2=10, B=1, lb=-1, ub=1)
@example(l0=0, l1=0, l2 =0,B =-0.1, lb=-0.02, ub=0)
@example(l0=0.0061658496635019775, l1=0.0, l2=1e-4, B=4, lb=0.0, ub=0.0015000000000000573)
def compare_B(l0, l1, l2, B, lb, ub):
    gurobi = calculate_optimal_gurobi(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
    gurobi = cost(B, gurobi, l0, l1, l2)
    closed = calculate_closed_form(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
    closed = cost(B, closed, l0, l1, l2)
    
    if gurobi <= closed:
        np.testing.assert_almost_equal(gurobi, closed)
    

In [6]:
compare_B()

Using license file /Users/tnonet/gurobi.lic
Academic license - for non-commercial use only


In [7]:
@given(l0=floats(0, 10), B=floats(-10, 10), lb=floats(-1, 0), ub=floats(0, 1))
@settings(max_examples=10000, deadline=None)
def compare_CDL0(l0, B, lb, ub):
    l1 = l2 = 0
    gurobi = calculate_optimal_gurobi(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
    gurobi = cost(B, gurobi, l0, l1, l2)
    closed = CDL0(l0=l0, B=B , lb=lb, ub=ub)
    closed = cost(B, closed, l0, l1, l2)
    
    if gurobi <= closed:
        np.testing.assert_almost_equal(gurobi, closed)
    

In [9]:
compare_CDL0()